## Covid19 Projection
The purpose of this notebook is to run Covid-19 case projections at State or Country levels. The outcome is the projection of the total confirmed cases for the target geography. 

This solution first tries to understand the approximate time to peak for the target entity (state/country), expected case rates and higher/lower bounds. It determines these parameters from countries that have exhibited similar trends in the past. 

Next, it determines the best simulation parameters in terms of transmission probability and weeks to peak by running the simulator iteratively with values from a parameter space.

Finally, it runs a simulation with the optimized parameters to generate day by day case projections. The simulation assumes 2 waves of infection surges following gaussian distribution and tries to apply that in generating the case projections. It also incorporates various factors like transmission probability, testing efficiency, intervention impacts etc.

Note: interventions_scorer.ipynb should be excuted prior to running the projections on new data

In [ ]:
!pip install -r requirements.txt

In [ ]:
import sys
sys.path.insert(1, 'src')
import pandas as pd
import numpy as np
import random
import config
import state_data_loader
import country_data_loader
import urllib.request
import os
# fixed_seed = 39
# random.seed(fixed_seed)
# np.random.seed(fixed_seed)

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from simulation_orchestrator import run

This simulation works best for a timeline when the infection is trending upward, i.e. it is able to find a greater than 1 transmission-rate and a postive time-to-peak from the recent data. As samples, we have used a few states from India and USA as the country for demonstration.

### Fetch latest data for target States / Countries

In [ ]:
# Set this flag to True if you want to download the latest COVID19 cases data from respective web sources
LOAD_LATEST_DATA = True

# * * * Run interventions_scorer.ipynb everytime you download the latest data.

In [ ]:
if LOAD_LATEST_DATA:
    
    # Function to refresh the local data file with the latest version from the web
    def download_latest_data (url, local_file):
        with urllib.request.urlopen(url) as response, open(local_file, 'wb') as out_file:
            data = response.read() # a `bytes` object
            out_file.write(data)
    
    # Mapping of online vs offline file locations to refresh
    online_offline_data = list()
    # Confirmed cases data maintained by Johns Hopkins University
    online_offline_data.append((config.confirmed_cases_global_online, 
                                os.path.join(config.base_data_dir, config.confirmed_cases_global_offline)))
    # Recovered cases data maintained by Johns Hopkins University
    online_offline_data.append((config.recovered_cases_global_online, 
                                os.path.join(config.base_data_dir, config.recovered_cases_global_offline)))
    # Deceased cases data maintained by Johns Hopkins University
    online_offline_data.append((config.deceased_cases_global_online, 
                                os.path.join(config.base_data_dir, config.deceased_cases_global_offline)))
    # Indian states specific cases maintained by COVID19INDIA (www.covid19india.org)
    online_offline_data.append((config.india_states_cases_online, 
                                os.path.join(config.base_data_dir, config.india_states_cases_offline)))
    
    # Refresh the local data files with the latest versions from respective web sources
    for path_pair in online_offline_data:
        try:
            download_latest_data (path_pair[0], path_pair[1])
            print ('Downloaded latest data from: {}'.format(path_pair[0]))
        except Exception as e:
            print ('Error while downloading {}: {}'.format(path_pair[0], e.__class__))
    
    # Transform and write the coutry specific data for further processing
    country_data_loader.load()
    
    target_states = ['KA', 'KL', 'MH', 'GJ', 'WB']
    # Transform and write the Indian states specific data for further processing
    state_data_loader.load('India', target_states)

In [ ]:
if LOAD_LATEST_DATA:
    target_states = ['KA', 'KL', 'MH', 'GJ', 'WB']
    # Transform and write the Indian states specific data for further processing
    state_data_loader.load('India', target_states)

    # For downloading US states
    # target_states = ['New York', 'California', 'Arizona', 'Alabama', 'New Jersey']
    # state_data_loader.load('US', target_states)

### Important parameters:
#### config.enable_case_rate_adjustment
- Setting this flag to True adjusts the infection case rate based on observations from countries with best matching case rate change pattern
- Setting this flag to False uses the infection case rate of the target state/country only

#### config.use_default_wave1_weeks_range
- Setting this flag to True will ensure that a default range (e.g. 1 to 5) is used instead of the weeks-to-peak values measured from other countries
- This could be set to False for most countries / states, except for the European countries where the cases peaked and declined in relatively short timespan

##### learn_params
- Whether to learn the optimal simulation parameters from the latest data, or to use the ones learnt during last optimization
- It's recommended to learn the parameters after fetching the latest confirmed-cases data

##### fitment_days
- Number of days of COVID-19 confirmed cases to use for learning / optimization the simulation parameters

##### test_days
- Latest n number of days to leave aside for testing
- Set this parameter to ensure that the simulation can learn from a period when the COVID19 case rate was increasing.
- Example 1: Let's say for a selected region COVID19 cases started to rise from 100 days before today and reached its peak in 30 days and declined to daily zero cases in another 30 days. Here, we can choose test_days = 75, as that will allow to learn the simulation parameters (fitment_days) from the growth phase of the infection. Alternatively, if we choose test_days = 5, we will try to learn simulation. parameters from a flat line and thus will get unexpected results.
- Example 2: Let's say for a selected region COVID19 cases started to rise from 60 days before today and are still rising. Here, we can choose test_days = 5, as that will allow to learn the simulation parameters (fitment_days) from the growth phase of the infection.

##### projection_days
- Number of days to project confirmed COVID19 cases for, including the test_days.

##### intv_inf_pctg
- Assumed influence of various interventions to reduce the spread of COVID19.
- should be between 0 (No influence) to 1 (Max influence)

#### country_code, state, state_population, actual_testing_capacity
- Target location specific parameters
- state, state_population parameters to represent the target country when projecting for a country

##### These parameters should be configured based on the target location and current day before running the projections.

*** Additional configurable parameters can be managed in src/config.python

In [ ]:
config.optimization_trials_low = 40

### Sample Projections for Indian States

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'IND', 'KL', 33406061, 6000
future_projection_days = 180

run (country_code, state, state_population, actual_testing_capacity, future_projection_days)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'IND', 'MH', 112374333, 50000
future_projection_days = 180

run (country_code, state, state_population, actual_testing_capacity, future_projection_days)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'IND', 'WB', 90305297, 10000
future_projection_days = 180

run (country_code, state, state_population, actual_testing_capacity, future_projection_days)

### Projections for Countries

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'USA', 'US', 311556874, 700000
future_projection_days = 90

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'COL', 'Colombia', 48901066, 4000
future_projection_days = 120

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'ARG', 'Argentina', 44044811, 4000
future_projection_days = 120

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'ITA', 'Italy', 59379449, 70000
future_projection_days = 90

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'NLD', 'Netherlands', 17231624, 20000
future_projection_days = 90

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'FRA', 'France', 66865144, 50000
future_projection_days = 90

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'PAK', 'Pakistan', 210000000, 90000
future_projection_days = 90

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'AUS', 'Australia', 24000000, 70000
future_projection_days = 90

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'BEL', 'Belgium', 11200000, 50000
future_projection_days = 90

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'DEU', 'Germany', 82200000, 150000
future_projection_days = 90

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'DNK', 'Denmark', 5600000, 40000
future_projection_days = 90

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'ESP', 'Spain', 46200000, 100000
future_projection_days = 90

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'GBR', 'United Kingdom', 65000000, 120000
future_projection_days = 90

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'CAN', 'Canada', 37000000, 90000
future_projection_days = 90

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'ISR', 'Israel', 8500000, 30000
future_projection_days = 180

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'JPN', 'Japan', 125000000, 150000
future_projection_days = 90

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'KOR', 'Korea, South', 50000000, 120000
future_projection_days = 90

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'MEX', 'Mexico', 120000000, 90000
future_projection_days = 180

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'PHL', 'Philippines', 105000000, 90000
future_projection_days = 180

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'QAT', 'Qatar', 2700000, 10000
future_projection_days = 90

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'RUS', 'Russia', 140000000, 150000
future_projection_days = 120

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'SAU', 'Saudi Arabia', 33000000, 90000
future_projection_days = 90

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'SGP', 'Singapore', 5500000, 20000
future_projection_days = 90

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'TUR', 'Turkey', 81000000, 90000
future_projection_days = 90

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'ZAF', 'South Africa', 57000000, 50000
future_projection_days = 120

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, country_level_projection=True)

In [ ]:
country_code, state, state_population, actual_testing_capacity = 'IND', 'India', 1300000000, 500000
future_projection_days = 180
init_infection = 3 #w.r.t. the configured simulation population size

run (country_code, state, state_population, actual_testing_capacity, future_projection_days, 
     country_level_projection=True, min_initial_infection = init_infection)